In [ ]:
import re
import pandas as pd

In [ ]:
with open('ep747b test.txt','r')as file:
    content = file.read()

In [ ]:
import re

def parse_settlement_report(filepath):
    data = {
        "reporting_for": None,
        "rollup_to": None,
        "proc_date": None,
        "report_date": None,
        "funds_xfer_entity": None,
        "settlement_currency": None,
        "clearing_currency": None,
        "transactions": []
    }

    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    for line in lines:
        if "REPORTING FOR" in line:
            match = re.search(r'REPORTING FOR:\s+(\d+)', line)
            if match:
                data["reporting_for"] = match.group(1)

        elif "ROLLUP TO" in line:
            match = re.search(r'ROLLUP TO:\s+(\d+)', line)
            if match:
                data["rollup_to"] = match.group(1)

        elif "PROC DATE" in line:
            match = re.search(r'PROC DATE:\s+(\d+)\w+\d+', line)
            if match:
                data["proc_date"] = match.group(1)
                print(data["proc_date"])

        elif "REPORT DATE" in line:
            match = re.search(r'REPORT DATE:\s+(\d+)', line)
            if match:
                data["report_date"] = match.group(1)

        elif "FUNDS XFER ENTITY" in line:
            match = re.search(r'FUNDS XFER ENTITY:\s+(\d+)', line)
            if match:
                data["funds_xfer_entity"] = match.group(1)

        elif "SETTLEMENT CURRENCY" in line:
            match = re.search(r'SETTLEMENT CURRENCY:\s+(\w+)', line)
            if match:
                data["settlement_currency"] = match.group(1)

        elif "CLEARING CURRENCY" in line:
            match = re.search(r'CLEARING CURRENCY:\s+(\w+)', line)
            if match:
                data["clearing_currency"] = match.group(1)

        elif re.search(r'\b(ORIGINAL SALE|ORIGINAL SALE RVRSL|DISPUTE|TOTAL|NET)\b', line):
            # Simple transaction line parsing
            parts = re.split(r'\s{2,}', line)
            if len(parts) >= 3:
                tx = {
                    "type": parts[0],
                    "Rating": None,
                    "count": None,
                    "clearing_amount": None,
                    "interchange_credits": None,
                    "interchange_debits": None
                }

                # Match trailing values based on pattern
                if re.search(r'\d+(\.\d{2})?(CR|DB)?$', line):
                    values = [v for v in parts[1:] if v]
                    if len(values) == 1:
                        tx["clearing_amount"] = values[0]
                    elif len(values) == 2:
                        tx['Rating'],tx["count"]= values
                    elif len(values) == 3:
                        tx['Rating'],tx["count"], tx["clearing_amount"]= values
                    elif len(values) == 4:
                        tx['Rating'],tx["count"], tx["clearing_amount"], tx["interchange_credits"] = values
                    elif len(values) >= 5:
                        tx['Rating'],tx["count"], tx["clearing_amount"], tx["interchange_credits"], tx["interchange_debits"] = values
                    data["transactions"].append(tx)

    return data

report = parse_settlement_report("ep747b test.txt")
print(report)

In [ ]:


# match_purchase =r'^\s+PURCHASE.*$\r\n\s+ORIGINAL SALE\s+([A-Z0-9]+)\s+(\d+)\s+([\d,]+\.\d{2}(?:CR|DR)?)\s+([\d,.]+)?'
# match_with_out_sale = r'^\s+([A-Z0-9]+)\s+(\d+)\s+([\d,]+\.\d{2}(?:DB|CR)?)\s+([\d,]+\.\d{2})'
# Split by the VSS report end marker
report_blocks = re.split(r'REPORT ID', content)
reports = []



reports = []
for block in report_blocks:
    if 'VSS-120' in block and 'VSS-950' not in block:
        lines = block.strip().split('\n')
        if len(lines) < 10:
            continue  # Skip invalid/empty sections

        # Extract report details
        report = {
            'ReportID': lines[0].split(':')[1].strip().split()[0],
            'ReportingFor': lines[1].split(':')[1].strip().split()[0],
            'RollupTo': lines[2].split(':')[1].strip().split()[0],
            'ProcDate': lines[1].split(':')[2].strip(),
            'ReportDate': lines[2].split(':')[2].strip(),
            'FundsXferEntity': lines[3].split(':')[1].strip().split()[0],
            'SettlementCurrency': lines[6].split(':', 1)[1].strip().split()[0] if len(lines) > 6 and ':' in lines[6] else '',
            'ClearingCurrency': lines[7].split(':')[1].strip() if len(lines) > 7 and ':' in lines[7] else None
        }



        transactions = []
        current_category = None  # <-- This will store "ACQUIRER TRANSACTIONS"
        current_subtype = None

        for line in lines:
            line = line.strip()
            if line and line.startswith('ACQUIRER TRANSACTIONS') and line.isupper():
                    Trans_type = line
                    
            elif line and line.startswith('ISSUER TRANSACTIONS') and line.isupper():
                    Trans_type = line

            # Capture top-level category like ACQUIRER TRANSACTIONS
            if line and line.startswith("TOTAL ORIGINAL SALE") and line.isupper():
                current_category =line.split()[0]+' '+line.split()[1]+' '+line.split()[2]
            
                match1 = re.match(r"TOTAL ORIGINAL SALE\s+([\d,]+)\s+([\d,]+\.\d{2}(?:CR|DB)?+)\s+([\d,]+\.\d{2})\s?",line)
                match2 =re.match(r"TOTAL ORIGINAL SALE\s+([\d,]+)\s+([\d,]+(?:CR|DB)?+)\s+([\d,]+\.\d+)",line)
                if match1:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': match1.group().split()[2],
                        'RateTableID': None,
                        'Count':(match1.group(1)),
                        'ClearingAmount': match1.group(2),
                        'InterchangeCredits': match1.group(3)
                    })
                                    
                if match2:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': match2.group().split()[2],
                        'RateTableID': None,
                        'Count':(match2.group(1)),
                        'ClearingAmount': match2.group(2),
                        'InterchangeCredits': match2.group(3)
                    })
            
            if line and line.startswith('TOTAL PURCHASE') and line.isupper():
                current_category = line.split()[0]+' '+line.split()[1]
                match_1 = re.match(r"TOTAL PURCHASE\s+([\d,]+)\s+([\d,]+\.\d+(?:CR|DB)?+)\s+([\d,]+\.\d)\s?",line)
                match_2 = re.match(r"TOTAL PURCHASE\s+([\d,]+)\s+([\d,]+(?:CR|DB)?+)\s+([\d,]+\.\d+)",line)
                #match_3 = re.match(r"TOTAL PURCHASE\s+([\d,]+)\s+([\d,]+(?:CR|DB)?+)\s+([\d,]+\.\d+)",line)
                if match_1:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': None ,
                        'RateTableID': None,
                        'Count':(match_1.group(1)),
                        'ClearingAmount': match_1.group(2),
                        'InterchangeCredits': match_1.group(3)
                    })
                                    
                if match_2:
                    transactions.append({
                        'Category': current_category,
                        'Subtype':None ,
                        'RateTableID': None,
                        'Count':(match_2.group(1)),
                        'ClearingAmount': match_2.group(2),
                        'InterchangeCredits': match_2.group(3)
                    })
                # if match_3:
                #     transactions.append({
                #         'Category': current_category,
                #         'Subtype':None ,
                #         'RateTableID': None,
                #         'Count':(match_3.group(1)),
                #         'ClearingAmount': match_3.group(2),
                #         'InterchangeCredits': match_3.group(3)
                #     })

            
            if line and line.startswith('TOTAL QUASI-CASH') and line.isupper():
                current_category = line.split()[0:2]
                quasi_match = re.match(r"TOTAL QUASI-CASH\s+([\d,]+)\s+([\d,]+\.\d+(CR|DB)+)\s+([\d,]+\.\d{2})\s?",line)
  
                if quasi_match:
                    transactions.append({
                        'Category': current_category,
                        'Subtype':None ,
                        'RateTableID': None,
                        'Count':(quasi_match.group(1)),
                        'ClearingAmount': quasi_match.group(2),
                        'InterchangeCredits': quasi_match.group(4)
                    })



                    
            if line and line.__contains__('RVRSL') and line.isupper():
                current_category = line.split()[0:2]
                match_rvrsl = re.match(r"^\w+\s+\w+\s+RVRSL\s+[\d,]+\s+([\d,]+\.\d+(?:CR|DB)?+)\s+([\d,]+\.\d+)",line)
                match_total_rvrsl = re.match(r'^\w+\s+\w+\s+\w+\s+RVRSL\s+[\d,]+\s+([\d,]+\.\d+(CR|DB)+)\s+([\d,]+\.\d+)',line)
                # print(current_category)    
                # print(match_total_rvrsl)  
                # print(match_rvrsl)         

                if match_rvrsl:
                    
                    transactions.append({
                        'Category': current_category,
                        'Subtype': match_rvrsl.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(match_rvrsl.group(0).split()[3]),
                        'ClearingAmount': match_rvrsl.group(1),
                        'InterchangeCredits': match_rvrsl.group(2)
                    })                
                
                if match_total_rvrsl:
                    
                    transactions.append({
                        'Category': current_category,
                        'Subtype': match_total_rvrsl.group(0).split()[2],
                        'RateTableID': None,
                        'Count':int(match_total_rvrsl.group(0).split()[4]),
                        'ClearingAmount': match_total_rvrsl.group(1),
                        'InterchangeCredits': match_total_rvrsl.group(3)
                    })          
            if line and line.startswith('TOTAL ATM CASH') and line.isupper():
                current_category = line.split()[0]+' '+line.split()[1]
                
                atm_match= re.match(r"TOTAL ATM CASH\s+\d+\s+([\d,]+\.\d+(CR|DB))\s+([\d,]+\.\d+)?",line)
                if atm_match:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': atm_match.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(atm_match.group(0).split()[3]),
                        'ClearingAmount': atm_match.group(1),
                        'InterchangeCredits': atm_match.group(3)
                    }) 


            if line and line.__contains__('MERCHANDISE CREDIT') and line.isupper():
                current_category=line.split()[0]+' '+line.split()[1]
                merchandise_match = re.match(r"TOTAL MERCHANDISE CREDIT\s+([\d,]+)\s+([\d,]+(?:CR|DB)?+)\s+([\d,]+\.\d+)?",line)
                
                #print(current_category)
                if merchandise_match:
                    line_parts = line.split()
                    transactions.append({
                        'Category': current_category,
                        'Subtype': merchandise_match.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(merchandise_match.group(1)),
                        'ClearingAmount':line.split()[4],
                        'InterchangeCredits': line.split()[5],
                        
                        'InterchangeDebits': line_parts[6] if len(line_parts) > 6 else None
                    })
                   # print(transactions)

                
            if line and line.startswith('TOTAL ISSUER INTERCHANGE') and line.isupper():
                current_category = line.split()[0]+' '+line.split()[1] +' '+line.split()[2]
                total_interchange = re.match(r"TOTAL ISSUER INTERCHANGE\s+([\d,]+)\s+([\d,]+\.\d+(CR|DB)+\s+([\d,]+\.\d+)+\s+([\d,]+\.\d+))?",line)
                total_interchange_2 = re.match(r"TOTAL ISSUER INTERCHANGE\s+([\d,]+)\s+([\d,]+(CR|DB)\s+([\d,]+\.\d+)\s+([\d,]+\.\d+))?",line)
               
                if total_interchange:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': total_interchange.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(total_interchange.group(1)),
                        'ClearingAmount':line.split()[4],
                        'InterchangeCredits': line.split()[5],
                        'InterchangeDebits': line.split()[6] if line.split()== [6] else None
                    })
                if total_interchange_2:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': total_interchange_2.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(total_interchange_2.group(1)),
                        'ClearingAmount':line.split()[4],
                        'InterchangeCredits': line.split()[5],
                        'InterchangeDebits': line.split()[6] if line.split()== [6] else None  #Invenstigate on how get the debits
                    })

            if line and line.startswith('TOTAL ACQUIRER INTERCHANGE') and line.isupper():
                current_category = line.split()[0]+' '+line.split()[1] +' '+line.split()[2]

                total_interchange3 = re.match(r"TOTAL ACQUIRER INTERCHANGE\s+([\d,]+)\s+([\d,]+\.\d+(CR|DB)+\s+([\d,]+\.\d+)+\s+([\d,]+\.\d+))?",line)
                total_interchange_4 = re.match(r"TOTAL ACQUIRER INTERCHANGE\s+([\d,]+)\s+([\d,]+(CR|DB)\s+([\d,]+\.\d+)\s+([\d,]+\.\d+))?",line)

                if total_interchange3:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': total_interchange3.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(total_interchange3.group(1)),
                        'ClearingAmount':line.split()[4],
                        'InterchangeCredits': line.split()[5],
                        'InterchangeDebits': line.split()[6] if line.split()== [6] else None
                    })
                if total_interchange_4:
                    transactions.append({
                        'Category': current_category,
                        'Subtype': total_interchange_4.group(0).split()[2],
                        'RateTableID': None,
                        'Count':(total_interchange_4.group(1)),
                        'ClearingAmount':line.split()[4],
                        'InterchangeCredits': line.split()[5],
                        'InterchangeDebits': line.split()[6] if line.split()== [6] else None  #Invenstigate on how get the debits
                    })

        reports.append(
            {
                "Report_details" : report,
                "Transaction Type" : Trans_type,
                "Transaction Data" : transactions
            }
        )
                
        print(reports)

In [ ]:
clearing_amount == 0
amt = 0
for trxn  in transactions:
    if trxn['ClearingAmount']:
        amt = trxn['ClearingAmount'].replace(',', '')
        if amt.endswith('CR') or amt.endswith('DB'):
            amt = amt[:-2]
            if amt.endswith('CR'):
                
                clearing_amount += float(amt) 
            if amt.endswith('DB'):
                clearing_amount -= float(amt)
    
    else:
        clearing_amount += 0
    print(clearint_amount)



In [ ]:
entries = []

for report in reports:
    details = report["Report_details"]
    reporting_for = details["ReportingFor"]
    rollup_to = details["RollupTo"]
    proc_date = details["ProcDate"]
    report_date = details["ReportDate"]
    funds_xfer_entity = details["FundsXferEntity"]
    settlement_currency = details["SettlementCurrency"]
    clearing_currency = details["ClearingCurrency"]

    print(f"reporting_for: {reporting_for}\nrollup_to: {rollup_to}\nproc_date: {proc_date}\nreport_date: {report_date}\nfunds_xfer_entity: {funds_xfer_entity}\nsettlement_currency: {settlement_currency}\nclearing_currency: {clearing_currency}")
    traxns_type = report["Transaction Type"]
    print(traxns_type)

            
    for txn in report["Transaction Data"]:
        transaction_type = txn["Category"]
        transaction_subtype = txn["Subtype"]
        rate_table_id = txn["RateTableID"]
        count = txn["Count"]

        clearing_amount = 0
        amt = txn.get('ClearingAmount', '').replace(',', '')

        if amt.endswith("CR"):
            clearing_amount = float(amt[:-2])
        elif amt.endswith("DB"):
            clearing_amount = -float(amt[:-2])

        interchange_credits = txn["InterchangeCredits"]
        #interchange_debits = txn["InterchangeDebits"]
        if interchange_credits.endswith("CR") or interchange_credits.endswith("DB"):
            interchange_credits = interchange_credits[:-2]
        # if interchange_debits.endswith("CR") or interchange_debits.endswith("DB"):
        #     interchange_debits = interchange_debits[:-2]

    

        entry = {
            "ReportID": details["ReportID"],
            "ReportingFor": reporting_for,
            "RollupTo": rollup_to,
            "ProcDate": proc_date,
            "ReportDate": report_date,
            "FundsXferEntity": funds_xfer_entity,
            "SettlementCurrency": settlement_currency,
            "ClearingCurrency": clearing_currency.split()[0] if clearing_currency else None,
            "Acq/Issuer": traxns_type,
            "Transaction Type": transaction_type,
            "Transaction Subtype": transaction_subtype,
            "RateTableID": rate_table_id,
            "Count": count,
            "ClearingAmount": clearing_amount,
            "InterchangeCredits": interchange_credits,
            #"InterchangeDebits": interchange_debits
        }

        entries.append(entry)

# Convert to DataFrame
df = pd.DataFrame(entries)

# Save to CSV
df.to_csv('output.csv', index=False)
#print(df)

# Save to Excel
df.to_excel('output.xlsx', index=False)


In [ ]:
import re
import pandas as pd
import logging
from tqdm import tqdm  # <-- Progress bar

# Setup logger
logging.basicConfig(filename='parse_vss_report.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def parse_vss_reports(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    reports = re.split(r"(?=REPORT ID:\s+(VSS-120|VSS-116))", content)
    extracted_rows = []
    unmatched_lines = []

    # Transaction category mapping based on your examples
    txn_category_mapping = {
        "QUASI-CASH": ["QUASI-CASH", "TOTAL QUASI-CASH"],
        "ORIGINAL SALE": ["TOTAL ORIGINAL SALE", "ORIGINAL SALE"],
        "PURCHASE": ["PURCHASE", "TOTAL PURCHASE", "NET PURCHASE"],
        "DISPUTE FIN": ["DISPUTE FIN"],
        "DISPUTE RESP FIN": ["DISPUTE RESP FIN"],
    }
    
    for report in tqdm(reports, desc="Parsing reports", unit="report"):
        if not report.strip().startswith("REPORT ID:  VSS-120"):
            continue

        # Extract metadata
        report_id_match = re.search(r"REPORT ID:\s+(VSS-120|VSS-116)", report)
        report_id = report_id_match.group(1) if report_id_match else "Unknown"

        logging.info(f"Processing report: {report_id}")

        reporting_for = re.search(r"REPORTING FOR:\s+(\d+)", report)
        reporting_for = reporting_for.group(1) if reporting_for else ""

        rollup_to = re.search(r"ROLLUP TO:\s+([A-Z0-9 ]+)", report)
        rollup_to = rollup_to.group(1).strip().split()[0] if rollup_to else ""

        funds_xfer = re.search(r"FUNDS XFER ENTITY:\s+(\d+)", report)
        funds_xfer = funds_xfer.group(1) if funds_xfer else ""

        proc_date = re.search(r"PROC DATE:\s+(\d{2}[A-Z]{3}\d{2})", report)
        proc_date = proc_date.group(1) if proc_date else ""

        report_date = re.search(r"REPORT DATE:\s+(\d{2}[A-Z]{3}\d{2})", report)
        report_date = report_date.group(1) if report_date else ""

        set_currency = re.search(r"SETTLEMENT CURRENCY:\s+([A-Z]+)", report)
        set_currency = set_currency.group(1) if set_currency else ""

        clear_currency = re.search(r"CLEARING CURRENCY:\s+([A-Z]+)", report)
        clear_currency = clear_currency.group(1) if clear_currency else ""

        business_type_issuer = re.search(r"ISSUER TRANSACTIONS",report)
        business_type_acquirer = re.search(r"ACQUIRER TRANSACTIONS",report)
        if business_type_acquirer:

            business_type = business_type_acquirer.group(0) 
        elif    business_type_issuer:
            business_type = business_type_issuer.group(0)
        else:
            pass

        #print(business_type)
        # Transaction blocks
                    
        txn_lines_1 = re.findall(
            r"^\s*(?:(?P<type>[A-Z \-]+(?:\s+[A-Z]+))\s+)?(?:(?P<rate>A\d{4})\s+)?(?P<count>[\d,]+)\s+(?P<amount>[0-9,]+(?:\.\d{2})?(?:CR|DB)?)?(?:\s+(?P<credit>[0-9,]+(?:\.\d{2})?))?(?:\s+(?P<debit>[0-9,]+(?:\.\d{2})?))?",
            report, re.MULTILINE)

        txn_lines_2 = re.findall(
            r"^\s*(?:(?P<type>[A-Z]+\s+))?(?:(?P<rate>A\d{4})\s+)?(?P<count>[\d,]+)\s+(?P<amount>[0-9,]+(?:\.\d{2})?(?:CR|DB)?)?(?:\s+(?P<credit>[0-9,]+(?:\.\d{2})?))?(?:\s+(?P<debit>[0-9,]+(?:\.\d{2})?))?",
            report, re.MULTILINE)
        


        if not txn_lines_1:
            logging.warning(f"No transaction lines found in report {report_id}")

        current_txn_type = ""
        for line in txn_lines_1:
            txn_type, rate_id, count, amount, credit, debit = line
            print(txn_type, rate_id, count, amount, credit, debit)
            if txn_type:
                current_txn_type = txn_type.strip()
            

            def parse_amount(val):
                if not val:
                    return 0.0
                val = val.replace(",", "")
                if val.endswith("CR"):
                    return float(val[:-2])
                elif val.endswith("DB"):
                    return -float(val[:-2])
                else:
                    return float(val)

            is_original_sale = "TOTAL ORIGINAL SALE" in txn_type.upper()

            if not count or not amount:
                unmatched_lines.append(line)
                logging.warning(f"Unmatched or incomplete line in report {report_id}: {line}")
                continue

            # Determine Transaction Category
            txn_category = "UNKNOWN"  # Default category if no match
            for category, txn_list in txn_category_mapping.items():
                if any(txn in txn_type.upper() for txn in txn_list):
                    txn_category = category
                    break

            extracted_rows.append({
                "Report ID": report_id,
                "Reporting For": reporting_for,
                "Rollup To": rollup_to,
                "Funds Xfer Entity": funds_xfer,
                "Proc Date": proc_date,
                "Report Date": report_date,
                "Settlement Currency": set_currency,
                "Clearing Currency": clear_currency,
                "Acq/Issuer": business_type,
                "Txn Type": current_txn_type,
                "Rate Table ID": rate_id.strip() if rate_id else "",
                "Count": (count),
                "Clearing Amount": parse_amount(amount),
                "Interchange Credit": float(credit.replace(",", "")) if credit else 0.0,
                "Interchange Debit": float(debit.replace(",", "")) if debit else 0.0,
                #"Transaction Category": tran_category,  # New column added for the category
                "Original Sale": is_original_sale
            })
            
 
        df = pd.DataFrame(extracted_rows)
        df.to_excel("parsed_vss_report_with_original_sale.xlsx", index=False, engine='openpyxl')
        print("Data inserted")

    if unmatched_lines:
        print("\n--- Unmatched Lines ---")
        for line in unmatched_lines:
            print(line)

    return df

# === USAGE ===
if __name__ == "__main__":
    input_path = "ep747b.txt"
    output_path = "parsed_vss_report_with vss120&vss160.xlsx"
    parse_vss_reports(input_path)


Parsing reports:   0%|          | 0/603 [00:00<?, ?report/s]